# Lab 03: TensorFlow vs. PyTorch
- Train a model on MNIST in both TensorFlow and PyTorch, convert to TFLite and ONNX.  
- Use tf.GradientTape for Tensorflow custom training loop.



## TensorFlow Implementation

In [8]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to range [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Build a simple feedforward neural network
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),             # Input shape matching MNIST image size
    tf.keras.layers.Flatten(),                         # Flatten 28x28 images to 1D vectors
    tf.keras.layers.Dense(64, activation='relu'),      # Hidden layer with 64 neurons and ReLU activation
    tf.keras.layers.Dense(10, activation='softmax')    # Output layer with 10 neurons (one per digit class)
])

# Compile the model with Adam optimizer and categorical crossentropy loss
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model and measure training time
start = time.time()
model.fit(x_train, y_train, epochs=5)
end = time.time()
print(f"TF Training time: {end-start:.2f} seconds")     # Print the training duration

# Evaluate the model on the test set
model.evaluate(x_test, y_test)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8608 - loss: 0.5039
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9541 - loss: 0.1576
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9677 - loss: 0.1075
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9769 - loss: 0.0798
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9808 - loss: 0.0627
TF Training time: 28.85 seconds
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9698 - loss: 0.1006


[0.08718627691268921, 0.9735000133514404]

## Convert TensorFlow model to TFLite

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmp7ylf53u5'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor_12')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  132712227649680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132712227649296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132712227639696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132712227649104: TensorSpec(shape=(), dtype=tf.resource, name=None)


## PyTorch Implementation

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

# Transform: Convert to tensor and flatten 28x28 image to a 784-dimensional vector
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # Flatten the image
])

# Data loaders with different batch sizes
train_loader = DataLoader(
    datasets.MNIST(root='./data', train=True, transform=transform, download=True),
    batch_size=64,  # Changed batch size
    shuffle=True
)
test_loader = DataLoader(
    datasets.MNIST(root='./data', train=False, transform=transform, download=True),
    batch_size=512  # Changed batch size
)

# Define a deeper fully connected neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)    # More neurons in first layer
        self.fc2 = nn.Linear(256, 64)     # Added a new hidden layer
        self.fc3 = nn.Linear(64, 10)      # Output layer

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))     # LeakyReLU instead of ReLU
        x = F.leaky_relu(self.fc2(x))
        return self.fc3(x)                # Final layer (logits)

# Instantiate model, loss function, and optimizer
model = Net()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.8)  # SGD instead of Adam

# Training loop
start = time.time()
for epoch in range(3):  # Fewer epochs
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
end = time.time()
print(f"PyTorch Training time: {end - start:.2f} seconds")

# Evaluation loop
model.eval()
correct = 0
with torch.no_grad():
    for x, y in test_loader:
        output = model(x)
        pred = output.argmax(1)
        correct += (pred == y).sum().item()
print(f"Test accuracy: {correct / len(test_loader.dataset):.4f}")


PyTorch Training time: 25.47 seconds
Test accuracy: 0.9748


## Convert PyTorch model to ONNX

In [11]:
# Install ONNX
!pip install onnx

In [12]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])

## TensorFlow custom training loop using tf.GradientTape

In [13]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Updated batch size
batch_size = 128

# Prepare datasets
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
    .shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define a deeper model with different activations
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='tanh'),     # First hidden layer with tanh
    tf.keras.layers.Dense(64, activation='relu'),      # Second hidden layer with ReLU
    tf.keras.layers.Dense(10, activation='softmax')    # Output layer
])

# Set up training components with RMSprop optimizer
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# Custom training loop
epochs = 3
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch, logits)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch + 1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Custom evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/3
Step 0, Loss: 2.3556, Accuracy: 0.1641
Step 100, Loss: 0.2793, Accuracy: 0.8417
Step 200, Loss: 0.2363, Accuracy: 0.8780
Step 300, Loss: 0.2403, Accuracy: 0.8938
Step 400, Loss: 0.1742, Accuracy: 0.9051
Training Accuracy for epoch 1: 0.9112

Epoch 2/3
Step 0, Loss: 0.2286, Accuracy: 0.9141
Step 100, Loss: 0.1542, Accuracy: 0.9492
Step 200, Loss: 0.2642, Accuracy: 0.9511
Step 300, Loss: 0.2085, Accuracy: 0.9527
Step 400, Loss: 0.1001, Accuracy: 0.9540
Training Accuracy for epoch 2: 0.9556

Epoch 3/3
Step 0, Loss: 0.1029, Accuracy: 0.9609
Step 100, Loss: 0.1288, Accuracy: 0.9666
Step 200, Loss: 0.1385, Accuracy: 0.9653
Step 300, Loss: 0.1125, Accuracy: 0.9659
Step 400, Loss: 0.1299, Accuracy: 0.9668
Training Accuracy for epoch 3: 0.9676

TF Training time: 58.99 seconds
Test Accuracy: 0.9647


## Performance Otimization with Graph Execution using @tf.function

In [14]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Updated batch size
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define a deeper model with LeakyReLU and ReLU
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Updated optimizer to SGD
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

@tf.function
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        loss = loss_fn(y_batch, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(y_batch, logits)
    return loss

# Training loop
epochs = 4
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss = train_step(x_batch, y_batch)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch + 1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(



Epoch 1/4
Step 0, Loss: 2.3168, Accuracy: 0.0781
Step 100, Loss: 1.5708, Accuracy: 0.4780
Step 200, Loss: 1.0091, Accuracy: 0.6074
Step 300, Loss: 0.7665, Accuracy: 0.6731
Step 400, Loss: 0.6660, Accuracy: 0.7127
Step 500, Loss: 0.7315, Accuracy: 0.7410
Step 600, Loss: 0.4417, Accuracy: 0.7625
Step 700, Loss: 0.3846, Accuracy: 0.7765
Step 800, Loss: 0.3200, Accuracy: 0.7898
Step 900, Loss: 0.3978, Accuracy: 0.8015
Training Accuracy for epoch 1: 0.8048

Epoch 2/4
Step 0, Loss: 0.2807, Accuracy: 0.9062
Step 100, Loss: 0.3891, Accuracy: 0.8926
Step 200, Loss: 0.3077, Accuracy: 0.8944
Step 300, Loss: 0.3790, Accuracy: 0.8950
Step 400, Loss: 0.5128, Accuracy: 0.8948
Step 500, Loss: 0.3437, Accuracy: 0.8959
Step 600, Loss: 0.4122, Accuracy: 0.8960
Step 700, Loss: 0.3774, Accuracy: 0.8963
Step 800, Loss: 0.4071, Accuracy: 0.8976
Step 900, Loss: 0.2390, Accuracy: 0.8991
Training Accuracy for epoch 2: 0.9001

Epoch 3/4
Step 0, Loss: 0.2972, Accuracy: 0.9062
Step 100, Loss: 0.2965, Accuracy: 0.